In [25]:
%pip install openai speechrecognition pyttsx3 pyaudio --quiet

Note: you may need to restart the kernel to use updated packages.


In [20]:
import getpass
import openai

openai.api_key = getpass.getpass("🔐 Enter your OpenAI API Key: ")
print("API key set successfully!")

API key set successfully!


In [ ]:
# Voice Input, GPT Reply, Voice Output
import speech_recognition as sr
import pyttsx3

# Initialize recognizer and voice engine
recognizer = sr.Recognizer()
engine = pyttsx3.init()

def listen():
    with sr.Microphone() as source:
        print("🎙️ Speak now...")
        engine.say("Speak now.")
        engine.runAndWait()

        try:
            audio = recognizer.listen(source, timeout=15)
        except sr.WaitTimeoutError:
            reminder_text = "I'm still listening. Please say something."
            print("🔔 Reminder: " + reminder_text)
            engine.say(reminder_text)
            engine.runAndWait()
            return ""  

    try:
        text = recognizer.recognize_google(audio)
        print(f"🗣️ You said: {text}")
        return text
    except sr.UnknownValueError:
        print("❌ Sorry, could not understand the audio.")
        return ""
    except Exception as e:
        print("❌ Error:", e)
        return ""


def ask_gpt(prompt):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4o-mini-2024-07-18",  
            messages=[{"role": "user", "content": prompt}]
        )
        answer = response["choices"][0]["message"]["content"]
        print(f"🤖 GPT: {answer}")
        return answer
    except Exception as e:
        print("❌ GPT error:", e)
        return ""

def speak(text):
    engine.say(text)
    engine.runAndWait()


In [ ]:
import csv
from datetime import datetime

# File path to store the conversation
log_file = "voice_chat_log.csv"

# Created CSV file with headers if not already created
def init_log():
    try:
        with open(log_file, "x", newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Timestamp", "User Input", "GPT Response"])
            print("📝 Log file created.")
    except FileExistsError:
        print("📄 Log file already exists. Appending data.")

# Append a row for each interaction
def log_to_csv(user_text, gpt_reply):
    with open(log_file, "a", newline='') as file:
        writer = csv.writer(file)
        writer.writerow([datetime.now().isoformat(), user_text, gpt_reply])
        print("✅ Interaction logged.")


In [ ]:
# Initialize log file
init_log()

# Main conversation loop
print("🔁 Start speaking! Say 'exit', 'quit', or 'stop' to end.")

while True:
    user_input = listen()

    if user_input.lower() in ["exit", "quit", "stop"]:
        goodbye_text = "Exiting the voice assistant. Goodbye!"
        print("👋", goodbye_text)
        speak(goodbye_text)
        break

    if user_input:
        gpt_reply = ask_gpt(user_input)
        speak(gpt_reply)
        log_to_csv(user_input, gpt_reply)
